## Install Spark and Initialize it

In [1]:
import findspark
findspark.init()

## Start Entry Points to Spark and import SQL functions

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, isnan, sum,collect_list,rank, max as sparkMax
from pyspark.sql.window import Window


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Read CSV, store as DF, print schema and get dimensions of dataset

In [3]:
df = spark.read \
            .option("inferSchema", "true") \
            .option("header", "true") \
            .csv("Aircraft_Wildlife_Strikes.csv") \
            .cache()

df.printSchema()
print("This dataset has", df.count(), "rows and", len(df.columns), "columns")

root
 |-- Record ID: integer (nullable = true)
 |-- Incident Year: integer (nullable = true)
 |-- Incident Month: integer (nullable = true)
 |-- Incident Day: integer (nullable = true)
 |-- Operator ID: string (nullable = true)
 |-- Operator: string (nullable = true)
 |-- Aircraft: string (nullable = true)
 |-- Aircraft Type: string (nullable = true)
 |-- Aircraft Make: string (nullable = true)
 |-- Aircraft Model: string (nullable = true)
 |-- Aircraft Mass: integer (nullable = true)
 |-- Engine Make: integer (nullable = true)
 |-- Engine Model: string (nullable = true)
 |-- Engines: integer (nullable = true)
 |-- Engine Type: string (nullable = true)
 |-- Engine1 Position: string (nullable = true)
 |-- Engine2 Position: integer (nullable = true)
 |-- Engine3 Position: string (nullable = true)
 |-- Engine4 Position: integer (nullable = true)
 |-- Airport ID: string (nullable = true)
 |-- Airport: string (nullable = true)
 |-- State: string (nullable = true)
 |-- FAA Region: string (nu

## Take a look at the first row of the dataset

In [4]:
display(df.take(1))

[Row(Record ID=127128, Incident Year=1990, Incident Month=1, Incident Day=1, Operator ID='DAL', Operator='DELTA AIR LINES', Aircraft='B-757-200', Aircraft Type='A', Aircraft Make='148', Aircraft Model='26', Aircraft Mass=4, Engine Make=34, Engine Model='40', Engines=2, Engine Type='D', Engine1 Position='1', Engine2 Position=1, Engine3 Position=None, Engine4 Position=None, Airport ID='KCVG', Airport='CINCINNATI/NORTHERN KENTUCKY INTL ARPT', State='KY', FAA Region='ASO', Warning Issued=None, Flight Phase='CLIMB', Visibility=None, Precipitation=None, Height=None, Speed=None, Distance=None, Species ID='NE1', Species Name='GULL', Species Quantity='1', Flight Impact=None, Fatalities=None, Injuries=None, Aircraft Damage=1, Radome Strike=1, Radome Damage=1, Windshield Strike=0, Windshield Damage=0, Nose Strike=0, Nose Damage=0, Engine1 Strike=0, Engine1 Damage=0, Engine2 Strike=0, Engine2 Damage=0, Engine3 Strike=0, Engine3 Damage=0, Engine4 Strike=0, Engine4 Damage=0, Engine Ingested=0, Prope

## Get a random sample of the data

In [5]:
display(df.sample(0.1).take(1))

[Row(Record ID=129780, Incident Year=1990, Incident Month=1, Incident Day=2, Operator ID='UNK', Operator='UNKNOWN', Aircraft='UNKNOWN', Aircraft Type=None, Aircraft Make=None, Aircraft Model=None, Aircraft Mass=None, Engine Make=None, Engine Model=None, Engines=None, Engine Type=None, Engine1 Position=None, Engine2 Position=None, Engine3 Position=None, Engine4 Position=None, Airport ID='PHLI', Airport='LIHUE ARPT', State='HI', FAA Region='AWP', Warning Issued=None, Flight Phase=None, Visibility=None, Precipitation=None, Height=None, Speed=None, Distance=0.0, Species ID='R1101', Species Name='BARN OWL', Species Quantity='1', Flight Impact=None, Fatalities=None, Injuries=None, Aircraft Damage=0, Radome Strike=0, Radome Damage=0, Windshield Strike=0, Windshield Damage=0, Nose Strike=0, Nose Damage=0, Engine1 Strike=0, Engine1 Damage=0, Engine2 Strike=0, Engine2 Damage=0, Engine3 Strike=0, Engine3 Damage=0, Engine4 Strike=0, Engine4 Damage=0, Engine Ingested=0, Propeller Strike=0, Propelle

### Get a cleaner view of the dataset using Pandas

In [6]:
df.toPandas()

,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,Aircraft Model,...,Fuselage Strike,Fuselage Damage,Landing Gear Strike,Landing Gear Damage,Tail Strike,Tail Damage,Lights Strike,Lights Damage,Other Strike,Other Damage
0,127128,1990,1,1,DAL,DELTA AIR LINES,B-757-200,A,148,26,...,0,0,0,0,1,1,0,0,0,0
1,129779,1990,1,1,HAL,HAWAIIAN AIR,DC-9,A,583,90,...,0,0,0,0,0,0,0,0,1,0
2,129780,1990,1,2,UNK,UNKNOWN,UNKNOWN,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,2258,1990,1,3,MIL,MILITARY,A-10A,A,345,None,...,0,0,0,0,0,0,0,0,0,0
4,2257,1990,1,3,MIL,MILITARY,F-16,A,561,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174099,366091,2015,9,30,SWA,SOUTHWEST AIRLINES,B-737-700,A,148,42,...,0,0,0,0,0,0,0,0,0,0
174100,365902,2015,9,30,SWA,SOUTHWEST AIRLINES,B-737-700,A,148,42,...,0,0,0,0,0,0,0,0,0,0
174101,365903,2015,9,30,BUS,BUSINESS,CITATIONJET,A,226,49,...,1,1,0,0,0,0,0,0,0,0
174102,366046,2015,9,30,QXE,HORIZON AIR,DHC8 DASH 8,A,303,10,...,1,0,0,0,0,0,0,0,0,0


## Data understanding and feature classification 

It seems that the ID feature corresponds to flights where different types of aircraft (civil, military or commercial) had a collision with a bird. There are some dimensions such as "Airport", "Operator", etc. and then we have dummy encoded variables with 0 and 1's to specify whether the described feature is true or not.

Therefore, after getting an idea of what the dataset is about and the different columns we have, we can categorize the features into the following 4 groups:
- **Timing related columns:** *Incident Year, Incident Month, Incident Day*
- **Flight and aircraft related columns:** *Operator ID, Operator, Aircraft, Aircraft Type, Aircraft Make, Aircraft Model, Aircraft Mass, Engine Make, Engine Model, Engines, Engine Type, Engine1 Position, Engine2 Position, Engine3 Position, Engine4 Position, Airport ID, Airport, State, FAA Region, Warning Issued, Flight Phase, Visibility, Precipitation, Height, Speed, Distance*
- **Bird related columns:** *Species ID, Species Name, Species Quantity*
- **Damage Evaluation columns:** *Flight Impact, Fatalities, Injuries, Aircraft Damage, Radome Strike, Radome Damage, Windshield Strike, Windshield Damage, Nose Strike, Nose Damage, Engine1 Strike, Engine1 Damage, Engine2 Strike, Engine2 Damage, Engine3 Strike, Engine3 Damage, Engine4 Strike, Engine4 Damage, Engine Ingested, Propeller Strike, Propeller Damage, Wing or Rotor Strike, Wing or Rotor Damage, Fuselage Strike, Fuselage Damage, Landing Gear Strike, Landing Gear Damage, Tail Strike, Tail Damage, Lights Strike, Lights Damage, Other Strike, Other Damage*

## Profiling of different categories

### A. For Time Related Columns:

In [7]:
print ("Summary of Time related columns:")
df.select("Incident Year","Incident Month","Incident Day").summary().show()
print("No NULLs detected")

Summary of Time related columns:
+-------+------------------+------------------+------------------+
|summary|     Incident Year|    Incident Month|      Incident Day|
+-------+------------------+------------------+------------------+
|  count|            174104|            174104|            174104|
|   mean|2006.0363920415384| 7.171839819877774|15.712263934200248|
| stddev| 6.747707856543079|2.7901520645834745| 8.799405130230447|
|    min|              1990|                 1|                 1|
|    25%|              2001|                 5|                 8|
|    50%|              2007|                 8|                16|
|    75%|              2012|                 9|                23|
|    max|              2015|                12|                31|
+-------+------------------+------------------+------------------+

No NULLs detected


In [8]:
print ("Most and least frequent occurrences for DayofMonth and DayOfWeek columns:")
dayofIncidenceOccurences = df.groupBy("Incident Day").count()
monthofIncidenceOccurences = df.groupBy("Incident Month").count()
yearofIncidenceOccurences = df.groupBy("Incident Year").count()


leastFreqDay = dayofIncidenceOccurences.orderBy(col("count").asc()).first()
mostFreqDay  = dayofIncidenceOccurences.orderBy(col("count").desc()).first()
leastFreqMonth = monthofIncidenceOccurences.orderBy(col("count").asc()).first()
mostFreqMonth  = monthofIncidenceOccurences.orderBy(col("count").desc()).first()
leastFreqYear = yearofIncidenceOccurences.orderBy(col("count").asc()).first()
mostFreqYear  = yearofIncidenceOccurences.orderBy(col("count").desc()).first()

display(Markdown("""
| %s | %s | %s | %s | %s | %s |
|----|----|----|----|----|----|
| %s | %s | %s | %s | %s | %s |
""" % ("leastFreqDay", "mostFreqDay", "leastFreqMonth", "mostFreqMonth","leastFreqYear", "mostFreqYear", \
       "%d (%d occurrences)" % (leastFreqDay["Incident Day"], leastFreqDay["count"]), \
       "%d (%d occurrences)" % (mostFreqDay["Incident Day"], mostFreqDay["count"]), \
       "%d (%d occurrences)" % (leastFreqMonth["Incident Month"], leastFreqMonth["count"]), \
       "%d (%d occurrences)" % (mostFreqMonth["Incident Month"], mostFreqMonth["count"]), \
       "%d (%d occurrences)" % (leastFreqYear["Incident Year"], leastFreqYear["count"]), \
       "%d (%d occurrences)" % (mostFreqYear["Incident Year"], mostFreqYear["count"]))))

Most and least frequent occurrences for DayofMonth and DayOfWeek columns:



| leastFreqDay | mostFreqDay | leastFreqMonth | mostFreqMonth | leastFreqYear | mostFreqYear |
|----|----|----|----|----|----|
| 31 (3151 occurrences) | 10 (6032 occurrences) | 2 (5587 occurrences) | 8 (24494 occurrences) | 1990 (2099 occurrences) | 2014 (13976 occurrences) |


Apparently 2014 was not a good year to travel by plane, there were almost 7 times more birdstrikes than in 1990

In [9]:
print("Checking amount of distinct values in Timing related columns:")
df.select([countDistinct(c).alias(c) for c in ["Incident Day","Incident Month","Incident Year"]]).show()

Checking amount of distinct values in Timing related columns:
+------------+--------------+-------------+
|Incident Day|Incident Month|Incident Year|
+------------+--------------+-------------+
|          31|            12|           26|
+------------+--------------+-------------+



According to the information above, we have an bird incident for 26 different years, and there are records for all 31 days of a month and for all months.
Besides this, there are no missing values in any of the time related columns

### B. For flight and aircraft related columns:

In [10]:
print ("Summary of Flight and Aircraft related columns:")
df.select("Operator ID", "Operator", "Aircraft", "Aircraft Type", "Aircraft Make").summary().show()
df.select("Aircraft Model", "Aircraft Mass", "Engine Make", "Engine Model", "Engines", "Engine Type").summary().show()
df.select("Engine1 Position", "Engine2 Position", "Engine3 Position", "Engine4 Position", "Airport ID").summary().show()
df.select("Airport", "State", "FAA Region", "Warning Issued", "Flight Phase", "Visibility", "Precipitation").summary().show()
df.select("Height", "Speed", "Distance").summary().show()

Summary of Flight and Aircraft related columns:
+-------+-----------+--------------------+------------+-------------+-----------------+
|summary|Operator ID|            Operator|    Aircraft|Aircraft Type|    Aircraft Make|
+-------+-----------+--------------------+------------+-------------+-----------------+
|  count|     174104|              174104|      174104|       133074|           131051|
|   mean|       null|                null|        null|         null|286.6323573244499|
| stddev|       null|                null|        null|         null|196.9489936339077|
|    min|       1AAH|         1US AIRWAYS|        A-10|            A|              04A|
|    25%|       null|                null|        null|         null|            148.0|
|    50%|       null|                null|        null|         null|            188.0|
|    75%|       null|                null|        null|         null|            345.0|
|    max|        ZAN|ZANTOP INTL AIRLINES|ZODIAC CH601|            J|   

In [11]:
print("Checking for nulls on Flight and Aircraft related columns:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Operator ID", "Operator", "Aircraft", "Aircraft Type",\
                                                              "Aircraft Make"]]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Aircraft Model", "Aircraft Mass", "Engine Make",\
                                                              "Engine Model", "Engines", "Engine Type"]]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Engine1 Position", "Engine2 Position", "Engine3 Position",\
                                                              "Engine4 Position", "Airport ID"]]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Airport", "State", "FAA Region", "Warning Issued",\
                                                              "Flight Phase", "Visibility"]]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in["Precipitation","Height", "Speed", "Distance"]]).show()

Checking for nulls on Flight and Aircraft related columns:
+-----------+--------+--------+-------------+-------------+
|Operator ID|Operator|Aircraft|Aircraft Type|Aircraft Make|
+-----------+--------+--------+-------------+-------------+
|          0|       0|       0|        41030|        43053|
+-----------+--------+--------+-------------+-------------+

+--------------+-------------+-----------+------------+-------+-----------+
|Aircraft Model|Aircraft Mass|Engine Make|Engine Model|Engines|Engine Type|
+--------------+-------------+-----------+------------+-------+-----------+
|         51665|        46784|      50670|       52116|  46762|      46822|
+--------------+-------------+-----------+------------+-------+-----------+

+----------------+----------------+----------------+----------------+----------+
|Engine1 Position|Engine2 Position|Engine3 Position|Engine4 Position|Airport ID|
+----------------+----------------+----------------+----------------+----------+
|           4791

As we can see, we have a lot of missing values in many columns but many of them make sense to be NULL. For example, there are many aircraft that only have 1 engine so in features such as **"Engine2 Position"**, **"Engine3 Position"** and **"Engine4 Position"** these single engine aircraft will have no values.

For **"Aircraft Type"**, **"Aircraft Make"** and **"Aircraft Model"** we can drop these features since all of them are mostly included in the **"Aircraft"** column. For example, in the first record we took from the dataset we had a B-757-200 which is a Boeing (Aircraft Make) and the model is B757-200 (Aircraft Model). Airplanes have a unique model code which is determined by the ICAO, special agency from the UN for International Civil Aircraft Organization.

On the other hand, we have several fields such as **"Visibility" , "Distance", "State", Flight Phase"** that have NULLs and therefore we'll have to evaluate whether we impute those missing values or if we drop those columns. Because we'll not be using these features for our business analysis, we can discard them.

---

As we can saw above, we have almost 22000 missing values for **State** but if we look at the **Airport** variable, we have only 290 missing values. Let's run a query to see which airports are those where **State** is NULL and the airport name is not "UNKNOWN":

In [12]:
df.select(["Airport","State"]).where((col("State").isNull()) & (col("Airport") != "UNKNOWN")).show()

+--------------------+-----+
|             Airport|State|
+--------------------+-----+
|     PUERTO VALLARTA| null|
|                ORLY| null|
| GUADALAJARA, MEXICO| null|
|SANTO DOMINGO/LAS AM| null|
|            NURNBERG| null|
|  HEWANORRA ST LUCIA| null|
|    GATWICK - LONDON| null|
|      TONCONTIN INTL| null|
|        REMOTE_WATER| null|
|        TOCUMEN INTL| null|
|      FRANKFURT MAIN| null|
|       FREEPORT INTL| null|
|  MINISTRO PISTARINI| null|
|        DHAHRAN INTL| null|
|     ST. LUCIA-VIGIE| null|
|        BERLIN TEGEL| null|
|        BERLIN TEGEL| null|
|GENERAL ESCOBEDO ...| null|
|   PUERTO PLATA INTL| null|
|     PUERTO VALLARTA| null|
+--------------------+-----+
only showing top 20 rows



Berlin, Gatwick, Guadalajara...This dataset is not just for bird airstrikes in the US, it's for all birdstrikes worldwide! As the database seems to have been compliled by the FAA (Federal Aviation Administration in the US), the **State** column is only applicable for US birdstrikes!
<br>
Now that we have this information, let's impute the missing values of the **State** feature where the **Airport** column is not "UNKNOWN" with something to indicate that the incident was abroad, not in the US:

In [13]:
df = df.withColumn("State", when(df.State.isNull(), lit("NOT USA")).otherwise(df.State))

In [14]:
df.toPandas()["State"]

0              KY
1              HI
2              HI
3              SC
4              FL
           ...   
174099    NOT USA
174100         TX
174101         FL
174102         OR
174103         GA
Name: State, Length: 174104, dtype: object

As we can see above, for row 174099, **"NOT USA"** has been properly imputed. Let's just do a simple count just in case:

In [15]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ["State"]]).show()

+-----+
|State|
+-----+
|    0|
+-----+



Perfect! No more NULL values in **State**.

---

To finish these set of columns, let's see the amount of distinct values we have in key variables we'll probably use later:

In [16]:
print("Checking amount of distinct values in Flight and Aircraft related columns:")
df.select([countDistinct(c).alias(c) for c in ["Operator","Aircraft","Engine Make","Airport","Flight Phase"]]).show()

Checking amount of distinct values in Flight and Aircraft related columns:
+--------+--------+-----------+-------+------------+
|Operator|Aircraft|Engine Make|Airport|Flight Phase|
+--------+--------+-----------+-------+------------+
|     533|     656|         34|   2226|          12|
+--------+--------+-----------+-------+------------+



We have incidents in 2226 different airports and for 656 different types of aircraft

### C. For Bird related columns:

In [17]:
print ("Summary of Bird related columns:")
df.select("Species ID", "Species Name", "Species Quantity").summary().show()

Summary of Bird related columns:
+-------+------------+------------------+----------------+
|summary|  Species ID|      Species Name|Species Quantity|
+-------+------------+------------------+----------------+
|  count|      174104|            174024|          169627|
|   mean|    Infinity|              null|             1.0|
| stddev|         NaN|              null|             0.0|
|    min|100000000000|ACADIAN FLYCATCHER|               1|
|    25%|    Infinity|              null|             1.0|
|    50%|    Infinity|              null|             1.0|
|    75%|    Infinity|              null|             1.0|
|    max|       k3317|      ZENAIDA DOVE|        Over 100|
+-------+------------+------------------+----------------+



**"Species ID"** has no missing values but columns **"Species Name"** and **"Species Quantity"** do have some although they are not many.

In [18]:
print ("Most and least frequent occurrences for Bird species:")
speciesIdOccurences = df.groupBy("Species ID").count()
speciesNameOccurences = df.groupBy("Species Name").count()

leastFreqSpecies = speciesIdOccurences.orderBy(col("count").asc()).first()
leastFreqName = speciesNameOccurences.orderBy(col("count").asc()).first()
mostFreqSpecies  = speciesIdOccurences.orderBy(col("count").desc()).first()
mostFreqName = speciesNameOccurences.orderBy(col("count").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqSpecies","leastFreqName", "mostFreqSpecies","mostFreqName",\
       "%s (%d occurrences)" % (leastFreqSpecies["Species ID"], leastFreqSpecies["count"]), \
       "%s" % (leastFreqName["Species Name"]), \
       "%s (%d occurrences)" % (mostFreqSpecies["Species ID"], mostFreqSpecies["count"]), \
       "%s" % (mostFreqName["Species Name"]))))

Most and least frequent occurrences for Bird species:



| leastFreqSpecies | leastFreqName | mostFreqSpecies | mostFreqName |
|----|----|----|----|
| YM1109 (1 occurrences) | CALLIOPE HUMMINGBIRD | UNKBM (38531 occurrences) | UNKNOWN MEDIUM BIRD |


The bird species mostly involved in air strikes is an unknown species of medium size while the least amount of reported incidents has been with the **Calliope Hummingbird**.

### D. For Damage Evaluation related columns:

In [19]:
print ("Summary of Damage Evaluation related columns:")
df.select("Flight Impact", "Fatalities", "Injuries", "Aircraft Damage").summary().show()
df.select("Radome Strike", "Radome Damage", "Windshield Strike", "Windshield Damage").summary().show()
df.select("Nose Strike", "Nose Damage", "Engine1 Strike", "Engine1 Damage").summary().show()
df.select("Engine2 Strike", "Engine2 Damage", "Engine3 Strike", "Engine3 Damage","Engine4 Strike", "Engine4 Damage").summary().show()
df.select("Engine Ingested", "Propeller Strike", "Propeller Damage", "Wing or Rotor Strike", "Wing or Rotor Damage", "Fuselage Strike").summary().show()
df.select("Fuselage Damage", "Landing Gear Strike", "Landing Gear Damage", "Tail Strike").summary().show()
df.select("Tail Damage", "Lights Strike", "Lights Damage", "Other Strike", "Other Damage").summary().show()

Summary of Damage Evaluation related columns:
+-------+--------------------+-------------------+------------------+------------------+
|summary|       Flight Impact|         Fatalities|          Injuries|   Aircraft Damage|
+-------+--------------------+-------------------+------------------+------------------+
|  count|               99465|                565|               229|            174104|
|   mean|                null|0.04601769911504425|1.7467248908296944|0.0859773468731333|
| stddev|                null|0.43516431196685945| 6.571239183141423|0.2803314004351369|
|    min|     ABORTED TAKEOFF|                  0|                 1|                 0|
|    25%|                null|                  0|                 1|                 0|
|    50%|                null|                  0|                 1|                 0|
|    75%|                null|                  0|                 1|                 0|
|    max|PRECAUTIONARY LAN...|                  8|              

With the mean of each dummy encoded column, we can already see a general overview of which type of damages occured the most. In most cases, the average is near 0 which makes sense because of all 174104 records, not all were specifically of one type of damage.

There are a lot of missing values for **Fatalities** and **Injuries**. Let's take a closer look at these 2 variables:

In [20]:
print("Checking for nulls on columns Fatalities and Injuries:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Fatalities","Injuries"]]).show()

Checking for nulls on columns Fatalities and Injuries:
+----------+--------+
|Fatalities|Injuries|
+----------+--------+
|    173539|  173875|
+----------+--------+



The amount of **Fatalities** and **Injuries** make sense for the type of strikes and the average aircraft damage (close to 0 or none) so for these 2 columns, we'll impute the missing values with zeros:

In [21]:
df = df.withColumn("Fatalities", when(df.Fatalities.isNull(), lit(0)).otherwise(df.Fatalities))

In [22]:
df = df.withColumn("Injuries", when(df.Injuries.isNull(), lit(0)).otherwise(df.Injuries))

Let's count the missing values for these columns once again:

In [23]:
print("Checking for nulls on columns Fatalities and Injuries:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Fatalities","Injuries"]]).show()

Checking for nulls on columns Fatalities and Injuries:
+----------+--------+
|Fatalities|Injuries|
+----------+--------+
|         0|       0|
+----------+--------+



Perfect! We got rid of those Null values!

## Get some business insights

For our analysis, we'll drop columns which don't add much value to because of redundancy, irrelevance or because of the high number of missing values and the difficulty to impute these missing values to columns that don't have a mean (categorical variables).

In [24]:
df = df.drop("Aircraft Type", "Aircraft Make","Aircraft Model","Height","Speed","Distance","Engine1 Position",
            "Engine2 Position","Engine3 Position","Engine4 Position","Engine Make","Warning Issued",
             "Precipitation","Aircraft Mass")

Our analysis will focus on the strikes themselves and the overall damage of the aircraft and not by parts such as wing damage, engine damage, etc. Therefore we'll drop all **damage** related features except for **Aircraft Damage**:

In [25]:
df = df.drop("Radome Damage","Windshield Damage","Nose Damage","Engine1 Damage","Engine2 Damage","Engine3 Damage",
            "Engine4 Damage","Propeller Damage","Wing or Rotor Damage","Fuselage Damage","Landing Gear Damage",
            "Tail Damage","Lights Damage","Other Damage")
len(df.columns)

38

Let's find out which **Aircraft** was the most involved in birdstrikes by aggregating each type of damage by **Aircraft**.

In [26]:
df_final = df.groupBy(["Aircraft"]).agg(
    sum("Aircraft Damage").alias("Aircraft_Damage"),sum("Engine Ingested").alias("Engine_Ingested"),\
    sum("Fatalities").alias("Fatalities"),sum("Injuries").alias("Injuries"),sum("Radome Strike").alias("Randome_Strikes"),\
    sum("Windshield Strike").alias("Windshield_Strikes"), sum("Nose Strike").alias("Nose_Strikes"),\
    sum("Engine1 Strike").alias("Engine1_Strikes"), sum("Engine2 Strike").alias("Engine2_Strikes"),\
    sum("Engine3 Strike").alias("Engine3_Strikes"), sum("Engine4 Strike").alias("Engine4_Strikes"),\
    sum("Propeller Strike").alias("Propeller_Strikes"),sum("Wing or Rotor Strike").alias("Wing_or_rotor_strikes"),\
    sum("Fuselage Strike").alias("Fuselage_Strikes"),sum("Landing Gear Strike").alias("Landing_Gear_Strikes"),\
    sum("Tail Strike").alias("Tail_Strikes"), sum("Lights Strike").alias("Lights_Strike"),\
    sum("Other Strike").alias("Other_Strikes")).orderBy(["Aircraft_Damage","Injuries"], ascending=[0,0])

In [27]:
df_final = df_final.withColumn("Total_Aircraft_Strikes",\
                            col("Randome_Strikes")+\
                            col("Windshield_Strikes")+col("Nose_Strikes")+\
                            col("Engine1_Strikes")+col("Engine2_Strikes")+col("Engine3_Strikes")+\
                            col("Engine4_Strikes")+col("Propeller_Strikes")+col("Wing_or_rotor_strikes")+\
                            col("Fuselage_Strikes")+col("Landing_Gear_Strikes")+col("Tail_Strikes")+\
                            col("Lights_Strike")+col("Other_Strikes"))

In [28]:
df_final = df_final.select("Aircraft", "Total_Aircraft_Strikes","Aircraft_Damage","Fatalities",\
                           "Injuries","Randome_Strikes", "Windshield_Strikes", "Nose_Strikes",\
                           "Engine1_Strikes", "Engine2_Strikes", "Engine3_Strikes",\
                           "Engine4_Strikes","Propeller_Strikes","Wing_or_rotor_strikes",\
                           "Fuselage_Strikes","Landing_Gear_Strikes","Tail_Strikes",\
                           "Lights_Strike", "Other_Strikes").orderBy(["Total_Aircraft_Strikes",
                                                                      "Aircraft_Damage"], ascending=[0,0])

In [29]:
df_final.toPandas().head(5)

,Aircraft,Total_Aircraft_Strikes,Aircraft_Damage,Fatalities,Injuries,Randome_Strikes,Windshield_Strikes,Nose_Strikes,Engine1_Strikes,Engine2_Strikes,Engine3_Strikes,Engine4_Strikes,Propeller_Strikes,Wing_or_rotor_strikes,Fuselage_Strikes,Landing_Gear_Strikes,Tail_Strikes,Lights_Strike,Other_Strikes
0,B-737-300,9957,975,0,2,1605,1429,1239,1118,982,0,0,0,1145,1376,345,140,54,524
1,B-737-700,8829,289,0,0,1508,1186,1125,668,581,0,0,0,1456,1197,338,105,35,630
2,A-320,7943,515,0,100,1370,1165,1141,482,477,0,0,2,747,1154,354,35,46,970
3,CRJ100/200,7236,380,0,0,548,1245,1551,171,173,0,0,1,1053,690,359,73,9,1363
4,B-757-200,6967,680,0,0,1233,1209,848,644,525,0,0,0,621,1009,282,26,41,529


As we can see above, after doing the aggregation and sorting our data in descending order first by **Total_Aircraft_Strikes** and then by **Aircraft_Damage** (to see how serious the impact was), the top 3 results are 2 Boeing aircraft (different variants of the popular 737) and in third place an Airbus A320. Let's categorize the occurence of bird strikes and see how many airplanes there are per category.

In [30]:
df_final = df_final\
   .withColumn("Strike_Occurence", when(col("Total_Aircraft_Strikes")<=2000,"1.very low")
                        .when((col("Total_Aircraft_Strikes")>2000) & (col("Total_Aircraft_Strikes")<=4000),"2.low")\
                        .when((col("Total_Aircraft_Strikes")>4000) & (col("Total_Aircraft_Strikes")<=6000),"3.medium")\
                        .when((col("Total_Aircraft_Strikes")>6000) & (col("Total_Aircraft_Strikes")<=8000),"4.high")\
                        .otherwise("5.very high"))\
    .cache()

df_Occ = df_final.select("Strike_Occurence", "Total_Aircraft_Strikes","Injuries","Fatalities","Aircraft")\
                     .groupBy("Strike_Occurence")\
                     .agg(count("Strike_Occurence").alias("Number_of_Aircraft_Involved"),
                         sum("Injuries").alias("Injuries"), sum("Fatalities").alias("Fatalities"))\
                     .orderBy("Strike_Occurence")\
                     .select("Strike_Occurence","Number_of_Aircraft_Involved","Injuries","Fatalities")
df_Occ.toPandas()

,Strike_Occurence,Number_of_Aircraft_Involved,Injuries,Fatalities
0,1.very low,637,251,23
1,2.low,11,39,3
2,3.medium,3,8,0
3,4.high,3,100,0
4,5.very high,2,2,0


There is a strange relationship as we can see since there are more injuries in the least occurent strike category rather than in the very high. Let's add the top aircraft per category of **Strike_Occurence** to see if you can make sense of this. We'll do it by grouping by strike category and aircraft and get the aircraft with the highest **Total_Aircraft_Strike** per category:

In [31]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

Sums = df_final.groupBy("Strike_Occurence", "Aircraft").agg(sum("Total_Aircraft_Strikes").alias("Total")).\
                                                                                            alias("Totals")

w = Window().partitionBy("Strike_Occurence").orderBy(col("Total").desc())

Sums_table = (Sums
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("Strike_Occurence","Aircraft")).orderBy("Strike_Occurence", ascending=[1,1])

summaryTable = df_Occ.join(Sums_table, on="Strike_Occurence").orderBy("Strike_Occurence", ascending=[1,1])

summaryTable.toPandas()

,Strike_Occurence,Number_of_Aircraft_Involved,Injuries,Fatalities,Aircraft
0,1.very low,637,251,23,EMB-170
1,2.low,11,39,3,B-727-200
2,3.medium,3,8,0,A-319
3,4.high,3,100,0,A-320
4,5.very high,2,2,0,B-737-300


This makes more sense now because we can see that when strike occurence is very low, the number of distinct aircraft is very high meaning that there is no one single plane that ALWAYS strikes birds, there are 637 of them. However, even though they have the lowest occurence, the have the most fatalities and injuries and this is mainly because amongst those 637 airplanes, there are some that are smaller regional jets with smaller turbines and a bird injested might damage the engines completely and others that are really small such as single engine Cessnas, Pipers, etc. Look at the top strikes in the very low category:

In [32]:
df_final.select(["Aircraft","Total_Aircraft_Strikes","Strike_Occurence","Injuries","Fatalities"]).\
        groupBy("Strike_Occurence","Aircraft").agg(sum("Total_Aircraft_Strikes").alias("Total_Aircraft_Strikes"),\
                                        sum("Injuries").alias("Injuries"),sum("Fatalities").alias("Fatalities")).\
        orderBy("Total_Aircraft_Strikes", ascending=[0,0]).where(col("Strike_Occurence") == "1.very low").toPandas().\
        head(30)

,Strike_Occurence,Aircraft,Total_Aircraft_Strikes,Injuries,Fatalities
0,1.very low,EMB-170,1727,0,0
1,1.very low,MD-11,1689,0,0
2,1.very low,BE-1900,1679,3,0
3,1.very low,B-717-200,1601,0,0
4,1.very low,MD-83,1600,0,0
5,1.very low,MD-88,1586,0,0
6,1.very low,UNKNOWN,1536,0,0
7,1.very low,B-737-400,1514,1,0
8,1.very low,EMB-135,1448,0,0
9,1.very low,DC-9-30,1447,0,0


On the other hand, the aircraft that have very high strike occurences are airliners such as Boeing's 737 (in fact, the only 2 aircraft in this strike category are different variants of the 737). It makes sense that this type of aircraft has the most occurences because they are used a lot by airlines and therefore the more time an aircraft has flying, the higher the chances of hitting a bird.

In [33]:
test=df_final.select(["Aircraft","Total_Aircraft_Strikes","Strike_Occurence","Injuries","Fatalities"]).\
        groupBy("Strike_Occurence","Aircraft").agg(sum("Total_Aircraft_Strikes").alias("Total_Aircraft_Strikes"),\
                                        sum("Injuries").alias("Injuries"),sum("Fatalities").alias("Fatalities"))
test.filter(test.Aircraft.like("%BE-35%")).toPandas()

,Strike_Occurence,Aircraft,Total_Aircraft_Strikes,Injuries,Fatalities
0,1.very low,BE-35,159,0,0
